<center><h2>Opal Notebook - Clustering<h2></center>

## Get Started

- Click *Cell* and *Run All* to kick it off (find it in the menu on top of this page) 
    - See the little circle on the upper right corner. It will be filled when the kernel is running and hollow when idle
- To restart everything, click *Kernel* and *Restart & Run All*
- Click *show/hide the raw code* button to show or hide the code
- Click *Insert* and *Insert Cell Above/Below* to insert a code block and start to write your own customized code (need to show code first)

In [5]:
# toggle code

from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to show/hide the raw code."></form>''')

In [6]:
# import os
# os.environ['defaultWebServer'] = 'https://webster.bfm.com'

In [7]:
# ## log process
# from blkbms.bms import BMS
# from blkbms.message import Message, SemaphoreMsgBody
# from blkcore.dates.utils import now
# import getpass
# user = getpass.getuser()

# keymap_pool = ['key_2', 'key_3', 'key_4', 'key_5',
#               'data_2', 'data_3', 'data_4', 'data_5']

# def pref_log(submodule, client='ADS', module='Opal-clustering', tag_def='version-1.0', user=user, **kwargs):
#     global keymap_pool
#     testmap = {'client': client,
#               'module': module,
#               'submodule': submodule,
#               'tag_def': tag_def,
#               'key_1': user}
    
#     for key, value in kwargs.items():
#         if key in keymap_pool:
#             testmap[key] = value
    
#     with BMS() as conn:    
#         bms_message = Message()
#         bms_message.init(SemaphoreMsgBody(testmap), 103)
#         bms_message.get_header().set_msg_subtype(7)
#         try:        
#             response = conn.send(bms_message, timeout=100)
#         except:
#             pass

# # logging
# pref_log(submodule='start_notebook')

In [8]:
# from blkcore.dates.holiday_calendar import HolidayCalendar
# from blkcore.dates.utils import today
# from blkcore.util import get_token

import pandas as pd
from pandas.tseries.offsets import BDay

from mltemplate.mldata import MLData
from mltemplate.mltsclustering import ClusterModel

import math
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.covariance import MinCovDet

from datetime import datetime
import copy

import matplotlib.pyplot as plt
plt.style.use('ggplot')
from matplotlib import pylab
%matplotlib inline

# %pylab inline
# pylab.rcParams['figure.figsize'] = (10, 6)

import colorlover as cl
from plotly.offline import init_notebook_mode, iplot
import plotly.plotly as py
import plotly.graph_objs as go
init_notebook_mode(connected=True)
from plotly.graph_objs import *

from ipywidgets import *
from IPython.display import display, clear_output, Image, Math, Latex

from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

# pd.set_option('display.max_rows', 10)
# pd.set_option('display.max_columns', 8)
# pd.set_option('display.max_colwidth', 10)

ModuleNotFoundError: No module named 'colorlover'

In [ ]:
mld = MLData(regret=True)
mld_time = MLData(regret=True)
mlc = ClusterModel()
module = HTML('')
chg_module = HTML('')
prepro_module = HTML('')
fill_na_submodule = HTML('')
cluster_module = HTML('')
cluster_submodule = HTML('')
operation = HTML('')
dim_module = HTML('')
cluster_info_module = HTML('')

data_dim_slice = pd.DataFrame()

ID_column_select = HTML('')
value_column_select = HTML('')
time_column_select = HTML('')
dim_column_select = HTML('')

In [ ]:
# display settings

# widgets
max_col = IntSlider(value=50, max=100)
max_row = IntSlider(value=15, max=200)
precision = IntSlider(value=4, max=10)
display(HBox([Label('Max # of Columns to Display:', layout=Layout(width='200px')),  max_col]))
display(HBox([Label('Max # of Rows to Display:', layout=Layout(width='200px')),  max_row]))
display(HBox([Label('Precision to display:', layout=Layout(width='200px')),  precision]))

# function
pd.options.display.max_columns = max_col.value
pd.options.display.max_rows = max_row.value
pd.options.display.precision = precision.value

def chg_setting(chg):
    pd.options.display.max_rows = max_row.value
    pd.options.display.max_columns = max_col.value
    pd.options.display.precision = precision.value

max_col.observe(chg_setting, names='value')
max_row.observe(chg_setting, names='value')
precision.observe(chg_setting, names='value')

# 1. Read Data

Enter the path to your data file to load the data. We also provide a sample data set in the below directory for users to do experiment. Copypaste the below file path to start! 

- Sample data path: data_sample/risk_data_science.csv

In [ ]:
# path_text = Text(layout=Layout(width='200px'))
# path_box = HBox([Label("Path:", layout=Layout(width='100px')), path_text])

filepath_label = Label("Filepath:", layout=Layout(width='100px'))
filepath_text = Text(layout=Layout(width='300px'))
filepath_box = HBox([filepath_label, filepath_text])

loaddata_button = Button(description="Load Data", button_style='success', layout=Layout(margin="20px 0px 0px 0px"))

load_progress_label = Label("Load Progress", layout=Layout(width='150px', margin="20px 0px 0px 10px"))
load_progress = IntProgress(value=0, min=0, max=10, step=1, bar_style='info', orientation='horizontal',
                            layout=Layout(width='150px', margin="20px 0px 0px 0px"))
load_progress_box = HBox([load_progress, load_progress_label])

load_box = VBox([filepath_box, loaddata_button, load_progress_box], layout=Layout(margin="20px 0px 0px 0px"))

display(load_box)

# functions
def loaddata(b):
    global load_box
    load_box.children[2].children[0].value = 3
    load_box.children[2].children[1].value = 'Loading Data...'
    clear_output()
    #filepath = os.path.join(path_text.value, filename_text.value)
    mld.read_data(load_box.children[0].children[1].value, infer_cat=False)

    load_box.children[2].children[0].value = 7
    display(mld.data)
    load_box.children[2].children[0].value = 10
    load_box.children[2].children[1].value = 'Data Loaded!'
    
    # logging
#     pref_log(submodule='loading_data')

load_box.children[1].on_click(loaddata)

# 2. Transpose Time-Series into columns

- Select ID: Specify ID column(s) in the dataframe. e.g. cusip, (cusip, purpose)
- Select Value: Choose the value column(s) for time-series. e.g. price, accr_int
- Select Time: Determine the column for time-index, only one column should be chosen. e.g. risk_date
- Select Dim: Select the dimensional information features. Not necessary for clustering model. e.g. purpose, currency, curve_type

In [ ]:
pivot_confirm_button = Button(description="Update Widget", button_style='Primary', layout=Layout(margin="20px 0px 0px 0px"))
display(pivot_confirm_button)

def pivot_confirm(b):
    clear_output()
    global module
   
    module.close()
    module = generate_pivot_table()
    display(module)

def do_pivot(b):
    clear_output()
    global mld_time, data_dim_slice, dim_column_select, data_time_dict
    global pivot_box
    
    module.children[2].children[0].value = 3
    module.children[2].children[1].value = 'Pivoting Data...'
    global cluster_info_module, need_update, analysis_need_update, analysis_old_display
    data_time_dict, data_dim_slice = mld.pivot_data_time(list(ID_column_select.value), [value_column_select.value],\
                                                    [time_column_select.value], list(dim_column_select.value))
    
    module.children[2].children[0].value = 7
    table_name = list(data_time_dict.keys())[0]
    mld_time = MLData(data_time_dict[table_name], regret=True)
    mld_time.save_current()
    #     data_preprocess = DataPreprocess(data_original=data_time_dict[table_name])
    #     data_time_dict_new = copy.deepcopy(data_time_dict)
    
    
    
    need_update = dict(zip(['Fill NA', 'Drop NA rows', 'Remove Holidays', 'Cap Data', 'Log Transform', 'Volatility Group'],
                       [True] * 6))
    
    analysis_need_update = dict(zip(['Plot TS', 'Plot Dim Info', 'Cluster Info', 'Detect Exception',
                                     'Individual Check', '2D Visualization', 'cluster_info_module'],
                                    [True] * 7))

    analysis_old_display = dict(zip(['Plot TS', 'Plot Dim Info', 'Cluster Info', 'Detect Exception',
                                     'Individual Check', '2D Visualization'],
                                    [None] * 6))
    
    cluster_info_module = update_cluster_info_module(cluster_info_module)
    
    module.children[2].children[0].value = 9
    for key in data_time_dict.keys():
        print('Time Series Table of {}: '.format(key))
        display(mld_time.data)

    if len(dim_column_select.value) == 0:
        print('You didn\'t choose any dimensional feature.')
    else:
        print('Dimensional feature table: ')

        display(data_dim_slice)
    
    module.children[2].children[0].value = 10
    module.children[2].children[1].value = 'Data Pivoted!'
    
    # logging
#     pref_log(submodule='pivot_table')
    
pivot_confirm_button.on_click(pivot_confirm)

In [ ]:
def get_value_dim_list(MLData_object):
    dtype_list = MLData_object.get_dtypes()
    dtype_dict = {}
    for feature in dtype_list:
        try:
            dtype_dict[dtype_list[feature]].append(feature)
        except KeyError:
            dtype_dict[dtype_list[feature]] = [feature]
    
    value_list = []
    for dtype_temp in ['int', 'float']:
        if dtype_temp in dtype_dict:
            value_list += dtype_dict[dtype_temp]
        else:
            pass
    
    dim_list = []
    for dtype_temp in ['category', 'object', 'bool']:
        if dtype_temp in dtype_dict:
            dim_list += dtype_dict[dtype_temp]
        else:
            pass
    
    return value_list, dim_list

In [ ]:
def generate_pivot_table():
    global ID_column_select, value_column_select, time_column_select, dim_column_select, pivot_box
    value_list, dim_list = get_value_dim_list(mld)
    
    ID_column_label = Label('Select ID(s)', layout=Layout(width='200px', margin="0px 0px 0px 0px"))
    ID_column_select = SelectMultiple(options=mld.data.columns.tolist(), layout=Layout(width='200px', margin="5px 0px 0px 0px"))
    ID_box = VBox([ID_column_label, ID_column_select])
    
    value_column_label = Label('Select Value', layout=Layout(width='200px', margin="0px 0px 0px 10px"))
    value_column_select = Dropdown(options=value_list, layout=Layout(width='200px', margin="5px 0px 0px 10px"))
    value_box = VBox([value_column_label, value_column_select])

    time_column_label = Label('Select Time', layout=Layout(width='200px', margin="0px 0px 0px 10px"))
    time_column_select = Dropdown(options=mld.data.columns.tolist(), layout=Layout(width='200px', margin="5px 0px 0px 10px"))
    time_box = VBox([time_column_label, time_column_select])
    
    dim_column_label = Label('Select Dim (Optional)', layout=Layout(width='200px', margin="0px 0px 0px 10px"))
    dim_column_select = SelectMultiple(options=dim_list, layout=Layout(width='200px', margin="5px 0px 0px 10px"))
    dim_box = VBox([dim_column_label, dim_column_select])
    
        
    do_pivot_button = Button(description="Pivot Table", button_style='success', layout=Layout(margin="30px 0px 0px 0px"))
    do_pivot_button.on_click(do_pivot)
    
    pivot_progress_label = Label("Pivot Progress", layout=Layout(width='150px', margin="20px 0px 0px 10px"))
    pivot_progress = IntProgress(value=0, min=0, max=10, step=1, bar_style='info', orientation='horizontal',
                                layout=Layout(width='150px', margin="20px 0px 0px 0px"))
    pivot_progress_box = HBox([pivot_progress, pivot_progress_label])
    
    box_layout = Layout(display='flex',
                        flex_flow='row',
                        align_items='baseline',
                        margin="10px 0px 0px 0px")
    
    pivot_box = HBox([ID_box, value_box, time_box, dim_box], layout=box_layout)
    module = VBox([pivot_box, do_pivot_button, pivot_progress_box], layout=Layout(margin="20px 0px 0px 0px"))
    return module

In [ ]:
def update_child(panel, idx, obj):

    ref = [list(panel.children)]
    for i, id in enumerate(idx[:-1]):
        ref.append(list(ref[i][id].children))

    ref[-1][idx[-1]] = obj
    for i in range(len(idx) - 1, 0, -1):
        ref[i - 1][idx[i - 1]].children = ref[i] 

    panel.children = ref[0]

In [ ]:
need_update = dict(zip(['Fill NA', 'Drop NA rows', 'Remove Holidays', 'Cap Data', 'Log Transform', 'Volatility Group'],
                       [False] * 6))

In [ ]:
def generate_fill_na_value_text():
    fill_na_text_label = Label('Fill NA value', layout=Layout(width='100px', margin='10px 0px 0px 0px'))
    fill_na_text = FloatText(value=0, layout=Layout(width='100px'))
    fill_na_value_text_box = VBox([fill_na_text_label, fill_na_text])
    return fill_na_value_text_box

def change_fill_na_method(chg):
    global fill_na_panel
    if fill_na_panel.children[1].value == 'value':
        fill_na_value_text_box = generate_fill_na_value_text()
        update_child(fill_na_panel, [3], fill_na_value_text_box)
    else:
        fill_na_panel.children[-1].close()
        fill_na_panel.children[-1].children[0].close()
        fill_na_panel.children[-1].children[1].close()

def update_fill_na_panel():
    global need_update
    if need_update['Fill NA']:
        fill_na_panel.children[1].value = fill_na_panel.children[1].options[0]
        fill_na_value_text_box = generate_fill_na_value_text()
        update_child(fill_na_panel, [3], fill_na_value_text_box)
        need_update['Fill NA'] = False
    return fill_na_panel

def fill_na(b):
    global fill_na_panel, mld_time
    clear_output()

    mld_time.fill_na_row(value=fill_na_panel.children[3].children[1].value, method=fill_na_panel.children[1].value)
    display(mld_time.data)
    
    # logging
#     pref_log(submodule='fill_na')

In [ ]:
fill_na_method_label = Label('Method', layout=Layout(margin='10px 0px 0px 0px'))
fill_na_method = Dropdown(options = ['value', 'mean', 'median', 'forward filling', 'backward filling'], value='value',
                         layout=Layout(width='100px'))

fill_na_method.observe(change_fill_na_method, names='value')
    
fill_na_button = Button(description='Fill NA', button_style='success', layout=Layout(margin='20px 0px 0px 0px'))

fill_na_panel = VBox([fill_na_method_label, fill_na_method, fill_na_button, generate_fill_na_value_text()])
fill_na_panel.children[2].on_click(fill_na)

In [ ]:
def update_rmv_na_panel():
    global need_update
    if need_update['Drop NA rows']:
        rmv_na_panel.children[1].value = 50
        need_update['Drop NA rows'] = False    
        
    return rmv_na_panel

def rmv_na(b):
    clear_output()
    mld_time.drop_na_row(thresh_slider.value)
    print('In total, {} ts data have been removed.'.format(mld_time.prev_data.shape[0] - mld_time.data.shape[0]))

    display(mld_time.data)
    
    # logging
#     pref_log(submodule='remove_na', key_2='percentage', data_2=float(thresh_slider.value))

In [ ]:
thresh_label = Label('Remove rows with NAs more than this percentage', layout=Layout(width='500px', margin="20px 0px 0px 0px"))
thresh_slider = IntSlider(min=0, max=100, value=50, layout=Layout(margin="20px 0px 0px 0px"))
rmv_na_button = Button(description="Remove NAs", button_style='success', layout=Layout(margin="20px 0px 0px 0px"))  

rmv_na_panel = VBox([thresh_label, thresh_slider, rmv_na_button])

rmv_na_panel.children[2].on_click(rmv_na)

In [ ]:
def remove_holidays(data, rmv_weekend=False, rmv_holiday=False, calendar_name='GreenPkg'):
    try:
        hc = HolidayCalendar(calendar_name)
    except Exception as ex:
        hc = HolidayCalendar('GreenPkg')
    column_date_list = [x.date() for x in data.columns]
    
    business_date_list = pd.date_range(start=column_date_list[0], end=column_date_list[-1], freq=BDay())
    business_date_list = [x.date() for x in business_date_list]
    
    rmv_weekend_list = list(map(lambda date: True if date in business_date_list else False, column_date_list))
    rmv_holiday_list = [not hc.is_holiday(x) for x in column_date_list]
    
    remain_date_list = [True] * len(column_date_list)
    
    if rmv_weekend:
        remain_date_list = [x and y for x, y in zip(rmv_weekend_list, remain_date_list)]
    if rmv_holiday:
        remain_date_list = [x and y for x, y in zip(rmv_holiday_list, remain_date_list)]
    
    return data.loc[:, remain_date_list]

In [ ]:
def update_rmv_holiday_panel():
    global need_update
    if need_update['Remove Holidays']:
        rmv_holiday_panel.children[0].children[1].value = False
        rmv_holiday_panel.children[1].children[1].value = False
        rmv_holiday_panel.children[1].children[2].children[1].value = 'GreenPkg'
        need_update['Remove Holidays'] = False    
    
    return rmv_holiday_panel

def rmv_hol(b):
    global rmv_holiday_panel, mld_time
    clear_output()
#     mld_time.remove_holidays(rmv_weekend=rmv_holiday_panel.children[0].children[1].value,
#                              rmv_holidays=rmv_holiday_panel.children[1].children[1].value)

    mld_time.prev_data = mld_time.data.copy(deep=True)
    data_rmv = remove_holidays(mld_time.data, rmv_weekend=rmv_holiday_panel.children[0].children[1].value,
                               rmv_holiday=rmv_holiday_panel.children[1].children[1].value,
                               calendar_name=rmv_holiday_panel.children[1].children[2].children[1].value)
    mld_time.data = data_rmv
    print('In total, {} timepoints have been removed.'.format(mld_time.prev_data.shape[1] - mld_time.data.shape[1]))
    display(mld_time.data)
    
    # logging
#     pref_log(submodule='remove_holiday')

In [ ]:
rmv_label = Label('Remove weekends', layout=Layout(width='150px', margin="20px 0px 0px 0px"))
weekend_check = Checkbox(value=False, layout=Layout(margin="20px 0px 0px 10px"))
weekend_box = HBox([rmv_label, weekend_check])

fed_label = Label('Remove holidays', layout=Layout(width='150px', margin="20px 0px 0px 0px"))
fed_hol_check = Checkbox(value=False, layout=Layout(margin="20px 0px 0px 10px"))

calendar_label = Label('Calendar', layout=Layout(width='100px', margin="20px 0px 0px 10px"))
calendar_text = Text(value='GreenPkg', layout=Layout(width='100px', margin="20px 0px 0px 10px"))
calendar_box = HBox([calendar_label, calendar_text])

holiday_box = HBox([fed_label, fed_hol_check, calendar_box])

rmv_hol_button = Button(description="Remove holidays", button_style='success', layout=Layout(margin="20px 0px 0px 0px"))

rmv_holiday_panel = VBox([weekend_box, holiday_box, rmv_hol_button])
    
rmv_holiday_panel.children[-1].on_click(rmv_hol)

In [ ]:
def update_cap_value_panel():
    global need_update
    if need_update['Cap Data']:
        cap_value_panel.children[0].children[1].value = 99
        cap_value_panel.children[1].children[1].value = 1                         
        need_update['Cap Data'] = False    
    return cap_value_panel

def cap_value(b):
    global mld, mld_time, cap_value_panel
    clear_output()
    print('Original Upper Bound: {}\nOriginal Lower Bound: {}'.format(mld_time.data.max().max(),
                                                                      mld_time.data.min().min()))
    mld_time.cap_data_time(upper_perc=cap_value_panel.children[0].children[1].value,
                           lower_perc=cap_value_panel.children[1].children[1].value)
    print('\nUpper Bound: {}\nLower Bound: {}'.format(mld_time.upper_b, mld_time.lower_b))
    display(mld_time.data)

    # logging
#     pref_log(submodule='cap_value')

In [ ]:
upper_label = Label('Cap data with percentile (Upper Bound)', layout=Layout(width='500px', margin="20px 0px 0px 0px"))
upper_bound_slider = IntSlider(min=0, max=100, value=99, layout=Layout(margin="20px 0px 0px 10px"))
upper_bound_box = HBox([upper_label, upper_bound_slider])

lower_label = Label('Cap data with percentile (Lower Bound)', layout=Layout(width='500px', margin="20px 0px 0px 0px"))
lower_bound_slider = IntSlider(min=0, max=100, value=1, layout=Layout(margin="20px 0px 0px 10px"))
lower_bound_box = HBox([lower_label, lower_bound_slider])

cap_button = Button(description="Cap value", button_style='success', layout=Layout(margin="20px 0px 0px 0px"))
   
cap_value_panel = VBox([upper_bound_box, lower_bound_box, cap_button]) 

cap_value_panel.children[2].on_click(cap_value)

In [ ]:
def update_log_value_panel():
    global need_update
    if need_update['Log Transform']:
        pass
        need_update['Log Transform'] = False    
    return log_value_panel


def log_value(b):
    global mld_time
    clear_output()   
    mld_time.log_transform()
    print('Log Transform the data.')
    display(mld_time.data)
    
    # logging
#     pref_log(submodule='log_transform')

In [ ]:
log_button = Button(description="Log value", button_style='success', margin="20px 0px 0px 0px")
log_value_panel = Box([log_button])

log_value_panel.children[0].on_click(log_value)

In [ ]:
def update_group_panel():
    global need_update
    if need_update['Volatility Group']:
        group_panel.children[0].children[1].value = 10
        group_panel.children[2].children[0].value = 0
        group_panel.children[2].children[1].value = 'Group Progress'
        need_update['Volatility Group'] = False
    return group_panel

def std_group(b):
    global mld_time, group_panel, data_dim_slice
    clear_output()
    group_panel.children[2].children[0].value = 3
    group_panel.children[2].children[1].value = 'Grouping by volatility...'

    #mld_time.get_robust_volatility(rmv_zero=False)
    if mld_time.data.isnull().sum().sum() > 0:
        print('There are missing values in the time-series data. We filled them with average value of each row.')
        group_panel.children[2].children[0].value = 5
        mld_time.fill_na_row()
    mld_time.group_by_std(n_quantiles=group_panel.children[0].children[1].value)
    group_panel.children[2].children[0].value = 7
    print('Group ts data by volatility. Add group ID as "std_group" column at last.')

    data_dim_slice['std_group'] = mld_time.data.loc[:, ['std_group']]
    
    column_list = mld_time.data.columns.tolist()
    col_0 = column_list.pop(-1)
    col_1 = column_list.pop(-1)
    column_list.insert(0, col_0)
    column_list.insert(0, col_1)

    display(mld_time.data.loc[:, column_list])
    #display(mld_time.data)
    group_panel.children[2].children[0].value = 10
    group_panel.children[2].children[1].value = 'Data Grouped!'
    
    # logging
#     pref_log(submodule='volatility_group', key_2='n_std_group', data_2=group_panel.children[0].children[1].value)

In [ ]:
group_label = Label('Number of Volatility group', layout=Layout(width='150px', margin="20px 0px 0px 0px"))
n_quantile_text = IntText(value=10, layout=Layout(width="50px", margin="10px 0px 0px 0px"))
group_box = VBox([group_label, n_quantile_text])
    
std_group_button = Button(description="Volatile Group", button_style='success', layout=Layout(margin="20px 0px 0px 0px"))

group_progress_label = Label("Group Progress", layout=Layout(width='150px', margin="20px 0px 0px 10px"))
group_progress = IntProgress(value=0, min=0, max=10, step=1, bar_style='info', orientation='horizontal',
                             layout=Layout(width='150px', margin="20px 0px 0px 0px"))
group_progress_box = HBox([group_progress, group_progress_label])
    
group_panel = VBox([group_box, std_group_button, group_progress_box])
   
group_panel.children[1].on_click(std_group)

In [ ]:
def undo_prepro(b):
    clear_output()

    mld_time.go_back()
    display(mld_time.data)
    
def retrieve_data(b):
    clear_output()
    mld_time.data = mld_time.data_snapshot.copy(deep=True)
    mld_time.save_current()
    print('original_data: ')
    display(mld_time.data)

In [ ]:
undo_button = Button(description="Undo", button_style='warning', layout=Layout(width='150px', margin="10px 0px 0px 16px"))
undo_button.on_click(undo_prepro)

retrieve_button = Button(description="Back to Original", button_style='danger',\
                         layout=Layout(width='150px', margin="10px 0px 0px 16px"))
retrieve_button.on_click(retrieve_data)

# 3. Data Preprocess

- Select the time-series that need to be preprocessed.
- Preprocess Functions:
  - Drop NA rows: Drop rows with missing values that exceed the threshold.
  - Fill NA: Fill NA with user-provided value or by specific method.
  - Remove Holidays: Determine whether to remove weekends or federal holidays.
  - Cap Data: Cap the data with specific qunatiles.
  - Log Transform: Transform the data with $f(x) = sign(x)\times\log(|x|+1)$
  - Volatility Group: Group the data based on their robust volatiles.
- Click on 'Undo' button to cancel the previous action.
- Click on 'Back to Original' to cancel all the actions on this time-series table.
- Click on 'Satisfied' when user belive the time-series data is ready to run the clustering model.


Note: The clustering model will not work if the data contains missing value. User may need to run 'Fill NA' or 'Drop NA rows' before clustering 

In [ ]:
preprocess_mapping = [update_rmv_na_panel, update_fill_na_panel, update_rmv_holiday_panel,\
                      update_cap_value_panel, update_log_value_panel, update_group_panel]
preprocess_tab = Tab(children=[rmv_na_panel, fill_na_panel, rmv_holiday_panel, cap_value_panel, log_value_panel, group_panel],
                    _titles={0: 'Drop NA rows', 1: 'Fill NA', 2: 'Remove Holidays', 3: 'Cap Data',\
                             4: 'Log Transform', 5: 'Volatility Group'},
                    layout=Layout(margin='20px 0px 0px 0px'))

def switch_preprocess(chg):
    '''Switch between processing functions.'''
    global mld_time
    clear_output()
    children = list(preprocess_tab.children)
    i = chg['new']
    children[i] = preprocess_mapping[i]()
    preprocess_tab.children = children
    if len(mld_time.data.columns.tolist()) > 0:     
        display(mld_time.data)
    else:
        pass

preprocess_tab.observe(switch_preprocess, names='selected_index')

display(preprocess_tab)
display(undo_button)
display(retrieve_button)

# 4. Build Clusters

In [ ]:
def clustering(b):
    clear_output()
    global cluster_label, data_dim_slice, cluster_info_module, mld_time, cluster_panel
    cluster_panel.children[3].children[0].value = 3
    cluster_panel.children[3].children[1].value = 'Clustering Time-Series Data...'
    if mld_time.data.isnull().sum().sum() > 0:
        print('There are missing values in the time-series data. We filled them with average value of each row.')
        mld_time.fill_na_row()
    
    if 'std_group' in mld_time.data.columns:
        cluster_label = mlc.cluster_model(mld_time.data, n_cluster_text.value, group_clustering=group_clustering_check.value)

    else:
        if group_clustering_check.value:
            print('TS data is not grouped by std yet. We will run the model in entire data universe.')
            cluster_label = mlc.cluster_model(mld_time.data, n_cluster_text.value, group_clustering=False)
        else:
            cluster_label = mlc.cluster_model(mld_time.data, n_cluster_text.value, group_clustering=False)
    #cluster_label['cluster_label'] = cluster_label['cluster_label'].astype('int')
    cluster_panel.children[3].children[0].value = 7

    data_dim_slice[cluster_label.columns[0]] = cluster_label
    
    if (group_clustering_check.value) and ('std_group' in data_dim_slice.columns):
        num_clusters_table = data_dim_slice.groupby(by = ['std_group'])['cluster_label'].nunique()
        num_clusters_table = pd.DataFrame(num_clusters_table)
        num_clusters_table.rename(columns={'cluster_label': '# clusters'}, inplace=True)
        display(num_clusters_table)
    
    cluster_info_module = update_cluster_info_module(cluster_info_module)
    
    display(data_dim_slice)
    cluster_panel.children[3].children[0].value = 10
    cluster_panel.children[3].children[1].value = 'Clustering Finished!'
    
    # logging
#     pref_log(submodule='run_clustering', key_2='n_cluster', data_2=float(n_cluster_text.value))
    
#     data_dim_copy = data_dim_slice.copy(deep=True)

In [ ]:
num_cluster_label = Label('Total number of clusters: ', layout=Layout(margin="20px 0px 0px 0px", width='150px'))
n_cluster_text = IntText(value=10, layout=Layout(margin="20px 0px 0px 20px", width='50px'))
num_cluster_box = HBox([num_cluster_label, n_cluster_text])

group_cluster_label = Label('Clustering in group', layout=Layout(margin="20px 0px 0px 0px", width='110px'))
group_clustering_check = Checkbox(value=False, layout=Layout(margin="20px 0px 0px 0px"))
group_cluster_box = HBox([group_cluster_label, group_clustering_check])

cluster_button = Button(description='Build Clusters', button_style='success',
                        layout=Layout(margin="20px 0px 0px 0px", width='150px'))

cluster_progress_label = Label("Cluster Progress", layout=Layout(width='150px', margin="20px 0px 0px 10px"))
cluster_progress = IntProgress(value=0, min=0, max=10, step=1, bar_style='info', orientation='horizontal',
                            layout=Layout(width='150px', margin="20px 0px 0px 0px"))
cluster_progress_box = HBox([cluster_progress, cluster_progress_label])

cluster_panel = VBox([group_cluster_box, num_cluster_box, cluster_button, cluster_progress_box])
cluster_panel.children[2].on_click(clustering)
display(cluster_panel)

In [ ]:
# data_dim_slice['cluster_label'] = list(map(int, data_dim_slice['cluster_label'].tolist()))

In [ ]:
analysis_need_update = dict(zip(['Plot TS', 'Plot Dim Info', 'Cluster Info', 'Detect Exception',
                                 'Individual Check', '2D Visualization', 'cluster_info_module'],
                       [False] * 7))

analysis_old_display = dict(zip(['Plot TS', 'Plot Dim Info', 'Cluster Info', 'Detect Exception',
                                 'Individual Check', '2D Visualization'],
                       [None] * 6))

In [ ]:
def update_cluster_info_module(cluster_info_module):
    global analysis_need_update
    try:
        cluster_info_module.children[0].children[1].options = data_dim_slice.columns.tolist()
        cluster_info_module.children[0].children[1].value = data_dim_slice.columns.tolist()[0]
    
        feature_temp = cluster_info_module.children[0].children[1].value
        cluster_value_option_list = list(data_dim_slice[feature_temp].unique())
        cluster_value_option_list = list(filter(lambda x: str(x) != 'nan', cluster_value_option_list))
        cluster_value_option_list.sort()
        cluster_info_module.children[1].children[1].options = cluster_value_option_list
        cluster_info_module.children[1].children[1].value = cluster_value_option_list[0]
    
        #cluster_info_module.children[1].children[1].value = ''
        cluster_info_module.children[2].children[1].value = False
        cluster_info_module.children[3].children[1].value = 'rmv_by_std'
        cluster_info_module.children[4].children[1].value = 2.0
        analysis_need_update['cluster_info_module'] = False
    except Exception as ex:
        analysis_need_update['cluster_info_module'] = False
    return cluster_info_module

In [ ]:
def change_cluster_value_option(chg):
    global data_dim_slice, cluster_info_module
    feature_temp = cluster_info_module.children[0].children[1].value
    cluster_value_option_list = list(data_dim_slice[feature_temp].unique())
    cluster_value_option_list = list(filter(lambda x: str(x) != 'nan', cluster_value_option_list))
    cluster_value_option_list.sort()
    cluster_info_module.children[1].children[1].options = cluster_value_option_list
    cluster_info_module.children[1].children[1].value = cluster_value_option_list[0]

In [ ]:
cluster_label_label = Label('Cluster By', layout=Layout(margin="20px 0px 0px 0px", width='150px')) 
cluster_label_dropdown = Dropdown(options=[],
                                  layout=Layout(width='100px', margin="20px 0px 0px 65px"))
cluster_label_box = HBox([cluster_label_label, cluster_label_dropdown])
    
cluster_id_label = Label('Select Cluster', layout=Layout(margin="20px 0px 0px 0px", width='150px')) 
cluster_id_text = Dropdown(options=[],
                           layout=Layout(width='100px', margin="20px 0px 0px 65px"))
cluster_id_box = HBox([cluster_id_label, cluster_id_text])

rmv_outlier_label = Label('Remove Outlier', layout=Layout(margin="20px 0px 0px 0px", width='150px'))
rmv_outlier_check = Checkbox(value=False, layout=Layout(margin="20px 0px 0px 0px"))
rmv_outlier_box = HBox([rmv_outlier_label, rmv_outlier_check])

rmv_method_label = Label('Method to remove outlier',  layout=Layout(margin="20px 0px 0px 0px", width='150px'))
method = RadioButtons(options={'Remove by standard devistion':'rmv_by_std', 'Novelty Detection': 'novelty_detection'},
                      value='rmv_by_std',
                      layout=Layout(margin="20px 0px 0px 65px"))
rmv_method_box = HBox([rmv_method_label, method])

std_label = Label('Standard Deviation Threshold', layout=Layout(margin="20px 0px 0px 0px", width='200px'))
std_slider = FloatSlider(min=1.0, max=10.0, value=2.0, layout=Layout(margin="20px 0px 0px 15px", width='200px'))
std_thresh_box = HBox([std_label, std_slider])
    
cluster_info_module = VBox([cluster_label_box, cluster_id_box, rmv_outlier_box, rmv_method_box, std_thresh_box])

In [ ]:
cluster_info_module.children[0].children[1].observe(change_cluster_value_option, names='value')

In [ ]:
#cluster_info_module = update_cluster_info_module(cluster_info_module)

In [ ]:
def change_cluster_id_value_type(cluster_label, cluster_id_value):
    mld_dim = MLData()
    mld_dim.data = data_dim_slice
    data_dim_slice_feature_type = mld_dim.get_dtypes()
    if data_dim_slice_feature_type[cluster_label] == 'int' or data_dim_slice_feature_type[cluster_label] == 'float':
        try:
            cluster_id_value_new = float(cluster_id_value)
        except ValueError:
            print('Input cluster id has a different datatype with cluster_label.')
    else:
        cluster_id_value_new = cluster_id_value
    return cluster_id_value_new

def plot_cluster_ts(b):
    global analysis_old_display
    clear_output()
    cluster_id_value_new = change_cluster_id_value_type(plot_ts_panel.children[0].children[0].children[1].value,
                                                        plot_ts_panel.children[0].children[1].children[1].value)
    ts_figure = mlc.plot_cluster_ts(label=cluster_id_value_new,
                                    data_time=mld_time.data,
                                    cluster_label=data_dim_slice[plot_ts_panel.children[0].children[0].children[1].value],
                                    data_original=mld_time.data_snapshot,
                                    mark_top_3=plot_ts_panel.children[1].children[0].children[1].value,
                                    rmv_outlier=plot_ts_panel.children[0].children[2].children[1].value,
                                    std_tol=plot_ts_panel.children[0].children[4].children[1].value,
                                    method=plot_ts_panel.children[0].children[3].children[1].value)
    analysis_old_display['Plot TS'] = ts_figure
    #display(analysis_old_display['Plot TS'].figure)
    
    # logging
#     pref_log(submodule='ts_plot')
    
def update_plot_ts_panel():
    global plot_ts_panel, cluster_info_module
    if analysis_need_update['Plot TS'] == True:
        if analysis_need_update['cluster_info_module'] == True:
            cluster_info_module = update_cluster_info_module(cluster_info_module)
            update_child(plot_ts_panel, [0, 0], cluster_info_module.children[0])
            update_child(plot_ts_panel, [0, 1], cluster_info_module.children[1])
            update_child(plot_ts_panel, [0, 2], cluster_info_module.children[2])
            update_child(plot_ts_panel, [0, 3], cluster_info_module.children[3])
            update_child(plot_ts_panel, [0, 4], cluster_info_module.children[4])
        
        plot_ts_panel.children[1].children[0].children[1].value = False
        analysis_need_update['Plot TS'] = False
    
    if analysis_old_display['Plot TS'] is not None:
        display(analysis_old_display['Plot TS'].figure)
    return plot_ts_panel

In [ ]:
mark_label = Label('Mark Top 3', layout=Layout(width='100px', margin="20px 0px 0px 0px"))
mark_check = Checkbox(value=False, layout=Layout(margin="20px 0px 0px 50px"))
    
marktop_box = HBox([mark_label, mark_check])
 
plot_button_ts = Button(description='Plot TS', button_style='success', layout=Layout(width='150px', margin="20px 0px 0px 0px"))


cluster_info_module_plot_ts = VBox([cluster_info_module.children[0],
                                    cluster_info_module.children[1],
                                    cluster_info_module.children[2],
                                    cluster_info_module.children[3],
                                    cluster_info_module.children[4]])

plot_ts_panel = HBox([cluster_info_module_plot_ts, VBox([marktop_box, plot_button_ts])])
plot_ts_panel.children[1].children[1].on_click(plot_cluster_ts)

In [ ]:
def plot_cluster_dim(b):
    global plot_dim_panel, analysis_old_display
    clear_output()
    cluster_id_value_new = change_cluster_id_value_type(plot_dim_panel.children[0].children[0].children[1].value,
                                                        plot_dim_panel.children[0].children[1].children[1].value)
    
    if plot_dim_panel.children[1].children[0].children[1].value == 'Single':
        feature_option_value = plot_dim_panel.children[1].children[1].children[1].value
    elif plot_dim_panel.children[1].children[0].children[1].value == 'Multiple':
        feature_option_value = list(plot_dim_panel.children[1].children[1].children[1].value)
    
    dim_figure = mlc.plot_cluster_dim(label=cluster_id_value_new,
                                      data_time=mld_time.data,
                                      cluster_label=data_dim_slice[plot_dim_panel.children[0].children[0].children[1].value],
                                      data_dim=data_dim_slice,
                                      feature=feature_option_value,
                                      rmv_outlier=plot_dim_panel.children[0].children[2].children[1].value,
                                      method=plot_dim_panel.children[0].children[3].children[1].value)
    analysis_old_display['Plot Dim Info'] = dim_figure

In [ ]:
def change_choice(chg):
    global plot_dim_panel, feature_option_box, feature_option_multi_box
    if plot_dim_panel.children[1].children[0].children[1].value == 'Single':
        update_child(plot_dim_panel, [1, 1], feature_option_box)
    elif plot_dim_panel.children[1].children[0].children[1].value == 'Multiple':
        update_child(plot_dim_panel, [1, 1], feature_option_multi_box)

In [ ]:
def update_plot_dim_panel():
    global plot_dim_panel, feature_option, feature_option_multi, cluster_info_module
    if analysis_need_update['Plot Dim Info'] == True:
        if analysis_need_update['cluster_info_module'] == True:
            cluster_info_module = update_cluster_info_module(cluster_info_module)
            update_child(plot_dim_panel, [0, 0], cluster_info_module.children[0])
            update_child(plot_dim_panel, [0, 1], cluster_info_module.children[1])
            update_child(plot_dim_panel, [0, 2], cluster_info_module.children[2])
            update_child(plot_dim_panel, [0, 3], cluster_info_module.children[3])
        
        plot_dim_panel.children[1].children[0].children[1].value = 'Single'
        feature_option.options = data_dim_slice.columns.tolist()
        feature_option_multi.options = data_dim_slice.columns.tolist()
        
        update_child(plot_dim_panel, [1, 1], feature_option_box)
        plot_dim_panel.children[1].children[0].children[1].observe(change_choice, names='value')
        
        analysis_need_update['Plot Dim Info'] = False
    
    if analysis_old_display['Plot Dim Info'] is not None:
        display(analysis_old_display['Plot Dim Info'].figure)
    return plot_dim_panel

In [ ]:
choice_label = Label('Choose', layout=Layout(width='150px', margin="20px 0px 0px 0px"))
choice_button = RadioButtons(options=['Single', 'Multiple'], value='Single', layout=Layout(margin="20px 0px 0px 65px"))
choice_box = HBox([choice_label, choice_button])

feature_option_label = Label('Feature', layout=Layout(width='150px', margin="20px 0px 0px 0px"))
feature_option = Dropdown(options=[], layout=Layout(width='100px', margin="20px 0px 0px 65px"))
feature_option_box = HBox([feature_option_label, feature_option])

feature_option_label_2 = Label('Features', layout=Layout(margin="20px 0px 0px 0px", width='150px'))
feature_option_multi = SelectMultiple(options=[],
                                      layout=Layout(width='100px', margin="20px 0px 0px 65px"))
feature_option_multi_box = HBox([feature_option_label_2, feature_option_multi])
    
plot_button_dim = Button(description='Plot Dim Info', button_style='success', layout=Layout(margin="20px 0px 0px 0px", width='150px'))

cluster_info_module_plot_dim = VBox([cluster_info_module.children[0],
                                     cluster_info_module.children[1],
                                     cluster_info_module.children[2],
                                     cluster_info_module.children[3]])


plot_dim_panel = HBox([cluster_info_module_plot_dim, VBox([choice_box, feature_option_box, plot_button_dim])])
#display(plot_dim_panel)

# plot_button_dim.on_click(plot_cluster_dim_single)
plot_dim_panel.children[1].children[2].on_click(plot_cluster_dim)
plot_dim_panel.children[1].children[0].children[1].observe(change_choice, names='value')

In [ ]:
def update_cluster_info_panel():
    global cluster_info_panel, feature_option_info, data_dim_slice, cluster_info_module
    if analysis_need_update['Cluster Info'] == True:
        if analysis_need_update['cluster_info_module'] == True:
            cluster_info_module = update_cluster_info_module(cluster_info_module)
            update_child(cluster_info_panel, [0, 0], cluster_info_module.children[0])
            update_child(cluster_info_panel, [0, 1], cluster_info_module.children[1])
            update_child(cluster_info_panel, [0, 2], cluster_info_module.children[2])
            update_child(cluster_info_panel, [0, 3], cluster_info_module.children[3])
        
        feature_option_info.options = data_dim_slice.columns.tolist()
        update_child(cluster_info_panel, [1, 0], feature_option_info_box)
        
        analysis_need_update['Cluster Info'] = False
    
    if analysis_old_display['Cluster Info'] is not None:
        display(analysis_old_display['Cluster Info'])
        
    return cluster_info_panel

In [ ]:
def change_order_of_info_table(info_table, feature_list):
    new_column_list = []
    new_column_list.append('n_points')
    new_column_list.append('mse')
    new_column_list.append('volatility')
    new_column_list.append('mse/volatility')
    
    for feature in feature_list:
        new_column_list.append(feature + '_entropy')
        for i in range(1, 4):
            new_column_list.append(feature + '_top_' + str(i))
    
    combined_entropy = list(set(info_table.columns.tolist()) - set(new_column_list))
    
    new_column_list = new_column_list + combined_entropy
    
    data_temp = info_table.loc[:, new_column_list]
    data_temp = data_temp.T
    data_temp.columns.name = 'cluster_id'
    
    return data_temp    

In [ ]:
def get_info(b):
    clear_output()
    cluster_id_value_new = change_cluster_id_value_type(cluster_info_panel.children[0].children[0].children[1].value,
                                                        cluster_info_panel.children[0].children[1].children[1].value)
    feature_list = list(cluster_info_panel.children[1].children[0].children[1].value)
    try:
        feature_list.remove('cluster_label')
    except ValueError:
        pass
    info_table, cover_list = mlc.get_cluster_info(label=cluster_id_value_new,
                                                  data_time=mld_time.data,
                                                  cluster_label=data_dim_slice[cluster_info_panel.children[0].children[0].children[1].value],
                                                  data_dim=data_dim_slice,
                                                  feature=feature_list, 
                                                  rmv_outlier=cluster_info_panel.children[0].children[2].children[1].value,
                                                  method=cluster_info_panel.children[0].children[3].children[1].value)
    info_table_new = change_order_of_info_table(info_table, feature_list)
    analysis_old_display['Cluster Info'] = info_table_new
    display(info_table_new)

In [ ]:
feature_option_label = Label('Features', layout=Layout(margin="20px 0px 0px 0px", width='150px'))
feature_option_info = SelectMultiple(options=[], layout=Layout(width='100px', margin="20px 0px 0px 65px"))
feature_option_info_box = HBox([feature_option_label, feature_option_info])
    
get_info_button = Button(description='Get Cluster Info', button_style='success', layout=Layout(margin="20px 0px 0px 0px", width='150px'))

cluster_info_module_cluster_info = VBox([cluster_info_module.children[0],
                                         cluster_info_module.children[1],
                                         cluster_info_module.children[2],
                                         cluster_info_module.children[3]])


cluster_info_panel = HBox([cluster_info_module_cluster_info, VBox([feature_option_info_box, get_info_button])])

# display(cluster_info_panel)

cluster_info_panel.children[1].children[1].on_click(get_info)

In [ ]:
def detect_exception(b):
    global exception_panel, analysis_old_display
    clear_output()

    cluster_id_value_new = change_cluster_id_value_type(exception_panel.children[0].children[0].children[1].value,
                                                        exception_panel.children[0].children[1].children[1].value)
    
    if exception_panel.children[1].children[0].children[1].children[0].value and exception_panel.children[1].children[1].children[1].children[0].value:
        abs_thresh = exception_panel.children[1].children[0].children[1].children[1].value
        rel_thresh = exception_panel.children[1].children[1].children[1].children[1].value/100.
    else:
        abs_thresh = False
        rel_thresh = False
    
    flag_table = mlc.generate_exception(label=cluster_id_value_new,
                                        data_time=mld_time.data,
                                        cluster_label=data_dim_slice[exception_panel.children[0].children[0].children[1].value],
                                        data_original=mld_time.data_snapshot,
                                        rmv_outlier=exception_panel.children[0].children[2].children[1].value,
                                        std_tol=exception_panel.children[0].children[4].children[1].value,
                                        method=exception_panel.children[0].children[3].children[1].value,
                                        abs_diff=abs_thresh,
                                        rel_diff=rel_thresh)
    try:
        flag_table.sort_values('# stds away', ascending=False, inplace=True)
        analysis_old_display['Detect Exception'] = flag_table
        # grid_exception = qgrid.QGridWidget(df = flag_table)
        # display(grid_exception)
        display(flag_table)
    except KeyError:
        analysis_old_display['Detect Exception'] = None
        print('No exception in this cluster.')
        
    # logging
    pref_log(submodule='detect_exception')

In [ ]:
def update_exception_panel():
    global exception_panel, grid_exception, cluster_info_module
    if analysis_need_update['Detect Exception'] == True:
        if analysis_need_update['cluster_info_module'] == True:
            cluster_info_module = update_cluster_info_module(cluster_info_module)
            update_child(exception_panel, [0, 0], cluster_info_module.children[0])
            update_child(exception_panel, [0, 1], cluster_info_module.children[1])
            update_child(exception_panel, [0, 2], cluster_info_module.children[2])
            update_child(exception_panel, [0, 3], cluster_info_module.children[3])
            update_child(exception_panel, [0, 4], cluster_info_module.children[4])
        
        exception_panel.children[1].children[0].children[1].children[0].value = False
        exception_panel.children[1].children[0].children[1].children[1].value = 0.5
        exception_panel.children[1].children[1].children[1].children[0].value = False
        exception_panel.children[1].children[1].children[1].children[1].value = 20
        analysis_need_update['Detect Exception'] = False
    
    if analysis_old_display['Detect Exception'] is not None:
        # grid_exception = qgrid.QGridWidget(df = analysis_old_display['Detect Exception'])
        # display(grid_exception)
        display(analysis_old_display['Detect Exception'])
        
    return exception_panel

In [ ]:
abs_label = Label('Absolute difference from cluster mean', layout=Layout(margin="20px 0px 0px 0px", width='250px'))
abs_diff_check = Checkbox(value=False, layout=Layout(margin="20px 0px 0px 0px"))
abs_diff_value = FloatText(value=0.5, layout=Layout(margin="20px 0px 0px 0px", width='50px'))
abs_diff_box = VBox([abs_label, HBox([abs_diff_check, abs_diff_value])]) 
    
rel_label = Label('Relative difference from cluster mean (%)', layout=Layout(margin="20px 0px 0px 0px", width='250px'))
rel_diff_check = Checkbox(value=False, layout=Layout(margin="20px 0px 0px 0px"))
rel_diff_value = IntSlider(min=0, max=100, value=20, layout=Layout(margin="20px 0px 0px 0px", width='200px'))
rel_diff_box = VBox([rel_label, HBox([rel_diff_check, rel_diff_value])])
    
exception_button = Button(description='Detect Exception', button_style='success', layout=Layout(margin="20px 0px 0px 0px", width='150px' ))

cluster_info_module_exception = VBox([cluster_info_module.children[0],
                                      cluster_info_module.children[1],
                                      cluster_info_module.children[2],
                                      cluster_info_module.children[3],
                                      cluster_info_module.children[4]])

exception_panel = HBox([cluster_info_module_exception, VBox([abs_diff_box, rel_diff_box, exception_button])])
exception_panel.children[1].children[2].on_click(detect_exception)

In [ ]:
def update_individual_check_panel():
    global individual_check_panel, ID_column_select, mld_time, time_column_select, cluster_info_module
    if analysis_need_update['Individual Check'] == True:
        if analysis_need_update['cluster_info_module'] == True:
            cluster_info_module = update_cluster_info_module(cluster_info_module)
            update_child(individual_check_panel, [0, 0], cluster_info_module.children[0])
            update_child(individual_check_panel, [0, 1], cluster_info_module.children[2])
            update_child(individual_check_panel, [0, 2], cluster_info_module.children[3])
            update_child(individual_check_panel, [0, 3], cluster_info_module.children[4])
        
        if ID_column_select.value and time_column_select.value and mld_time:
            ID_BOX_list_temp = []
            for ID_column in ID_column_select.value:
                initial_id_label_temp = Label(ID_column, layout=Layout(width='100px', margin="20px 0px 0px 0px"))
                id_text_temp = Text(value='', layout=Layout(width='150px', margin="20px 0px 0px 65px")) # should detect dtype, maybe not string
                ID_BOX_list_temp.append(HBox([initial_id_label_temp, id_text_temp]))
            ID_BOX_temp = VBox(ID_BOX_list_temp)
            
            time_list = mld_time.data.columns.tolist()
            try:
                time_list.remove('robust_std')
            except ValueError:
                pass
            try:
                time_list.remove('std_group')
            except ValueError:
                pass
            try:
                time_list.remove('cluster_label')
            except ValueError:
                pass
            
            mld_time.time_list = time_list
            
            date_select_label_temp = Label(time_column_select.value, layout=Layout(width='100px', margin="20px 0px 0px 0px"))
            date_select_temp = Dropdown(options=time_list, layout=Layout(width='150px', margin="20px 0px 0px 65px"))
            date_select_box = HBox([date_select_label_temp, date_select_temp])
            
            update_child(individual_check_panel, [1, 0], ID_BOX_temp)
            update_child(individual_check_panel, [1, 1], date_select_box)
            
            analysis_need_update['Individual Check'] = False
            
    if analysis_old_display['Individual Check'] is not None:
        for fig_temp in analysis_old_display['Individual Check']:
            iplot(fig_temp)
    return individual_check_panel

In [ ]:
def Check(b):
    clear_output()
    global index_name, time, individual_check_panel, analysis_old_display
    index_name = []
    for text in individual_check_panel.children[1].children[0].children:
        index_name.append(text.children[1].value)
    index_name = tuple(index_name)
    if len(index_name) == 1:
        index_name = index_name[0]

    time = individual_check_panel.children[1].children[1].children[1].value
    time = pd.to_datetime(time)
    change_into_datetime_object(mld_time)
    fig_1, fig_2 = check_all(index_name, time)
    analysis_old_display['Individual Check'] = [fig_1, fig_2]
    
    for fig in analysis_old_display['Individual Check']:
        iplot(fig)

In [ ]:
def check_all(index_name, time):
    print('Summary: ')
    global individual_check_panel
    try:
        if isinstance(index_name, tuple):
            for i in range(len(ID_column_select.value)):
                print(ID_column_select.value[i] + ': {}'.format(index_name[i]))
        elif isinstance(index_name, str):
            print(ID_column_select.value[0] + ': {}'.format(index_name))
        
        print(data_dim_slice.loc[index_name, :])
            
        if len(mld_time.data.loc[[index_name], :]) > 0:
            cluster_label_list = data_dim_slice.loc[[index_name],\
                                                    individual_check_panel.children[0].children[0].children[1].value].unique()
            if len(cluster_label_list) > 1:
                return print('This time-series belongs to multiple clusters.')
            else:
                cluster_id_value_new = change_cluster_id_value_type(individual_check_panel.children[0].children[0].children[1].value,\
                                                                    cluster_label_list[0])
                if individual_check_panel.children[0].children[1].children[1].value:
                    data_time_rmv_outlier, data_dim_rmv_outlier = \
                    mlc.remove_outlier(label=cluster_id_value_new,
                                       data_time=mld_time.data,
                                       cluster_label=data_dim_slice[individual_check_panel.children[0].children[0].children[1].value],
                                       data_dim=data_dim_slice,
                                       method=individual_check_panel.children[0].children[2].children[1].value)
                else:
                    data_time_rmv_outlier, data_dim_rmv_outlier = \
                    mlc.extract_info_from_cluster(label=cluster_id_value_new,
                                                  data_time=mld_time.data,
                                                  cluster_label=data_dim_slice[individual_check_panel.children[0].children[0].children[1].value],
                                                  data_dim=data_dim_slice)
    
            if len(data_time_rmv_outlier.loc[[index_name], :]) > 0:
                print('\n{} belongs to cluster {}.'.format(index_name, cluster_id_value_new))
    except KeyError:
        return print('{} is not covered by clustering model.'.format(index_name))
    
    exception_value_original = mld_time.data_snapshot.loc[index_name, time]
    exception_value = mld_time.data.loc[index_name, time]

    temp_logged = data_time_rmv_outlier.loc[:, [time]]
    loc_logged, var_logged, std_logged = cluster_robust_mean_std(temp_logged)

    upper_bound = loc_logged[0][0] + individual_check_panel.children[0].children[3].children[1].value*std_logged[0][0]
    lower_bound = loc_logged[0][0] - individual_check_panel.children[0].children[3].children[1].value*std_logged[0][0]

    exception_result = False
    if (exception_value > upper_bound) or (exception_value < lower_bound):
        exception_result = True
    
    y = mld_time.data_snapshot.loc[index_name, mld_time.time_list]
    original_value = go.Scatter(x=mld_time.time_list, y=y.values.tolist(), name = 'value original')

    exception_point = go.Scatter(x = [time], y = [exception_value_original], mode = 'markers', marker = dict(size=10, color='green'),
                                 name = 'Point')

    data = [original_value, exception_point]
    layout = dict(title = '{}'.format(index_name),
                  xaxis=dict(
                    zeroline=False
                    ),
                  yaxis=dict(
                    zeroline=False,
                    ))
    fig_1 = go.Figure(data=data, layout=layout)
    #iplot(fig_1)

    if exception_result:
        print('{} is flagged as an exception on {}/{}/{}.'.format(index_name, time.month, time.day, time.year))
        point_name = 'Exception'

    else:
        print('{} is not an exception on {}/{}/{}.'.format(index_name, time.month, time.day, time.year))
        point_name = 'Not Exception'
    
    cluster_value = []
    index_list = data_time_rmv_outlier.index.tolist()

    location_list, square_error_list, std_list = cluster_robust_mean_std(data_time_rmv_outlier)
    upper_bound = location_list + individual_check_panel.children[0].children[3].children[1].value * std_list
    lower_bound = location_list - individual_check_panel.children[0].children[3].children[1].value * std_list

    location_list = location_list.reshape(len(location_list))
    upper_bound = upper_bound.reshape(len(upper_bound))
    lower_bound = lower_bound.reshape(len(lower_bound))
    lower_bound_rev = lower_bound[::-1]

    go_center = go.Scatter(
            x=mld_time.time_list,
            y=location_list,
            name = 'Cluster Center',
            line = dict(
                color = 'red',
                width = 2,
                dash = 'dash')
        )

    cluster_value.append(go_center)

    for port in index_list:
        if port == index_name:
            go_temp = go.Scatter(
            x=mld_time.time_list,
            y=data_time_rmv_outlier.loc[port],
            name = str(index_name),
            line = dict(
                color = 'black',
                width = 1),
            showlegend=True,
        )
            cluster_value.append(go_temp)
        else:
            continue

    time_rev = mld_time.time_list[::-1]
    std_band = go.Scatter(
        x = mld_time.time_list + time_rev,
        y = list(upper_bound) + list(lower_bound_rev),
        fill='tozerox',
        fillcolor = 'rgba(231,107,243,0.2)',
        line=Line(color='transparent'),
        name='Band',
    )


    exception_value = mld_time.data.loc[index_name, time]
    exception_point = go.Scatter(x = [time], y = [exception_value], mode = 'markers', marker = dict(size=10), name = point_name)
    cluster_value.append(exception_point)
    cluster_value.append(std_band)
    
    #     lower_bound = data_time_rmv_outlier.min().min()
    #     upper_bound = data_time_rmv_outlier.max().max()
    
    layout = go.Layout(
        xaxis=dict(
        zeroline=False
    ),
        
        yaxis=dict(
        zeroline=False,
    )
    )

    fig_2 = go.Figure(data=cluster_value, layout=layout)
    #iplot(fig_2)
    return fig_1, fig_2

In [ ]:
ID_BOX_list = []

initial_id_label = Label('TS id', layout=Layout(width='100px', margin="20px 0px 0px 0px"))
initial_id_text = Text(value='', layout=Layout(width='100px', margin="20px 0px 0px 65px"))

ID_BOX_list.append(HBox([initial_id_label, initial_id_text]))

ID_BOX = VBox(ID_BOX_list)

date_select_label = Label('Time', layout=Layout(width='100px', margin="20px 0px 0px 0px"))
date_select = Dropdown(options=[], layout=Layout(width='100px', margin="20px 0px 0px 65px"))

date_select_box = HBox([date_select_label, date_select])

check_button = Button(description="Check!", button_style='success', layout=Layout(margin="20px 0px 0px 0px"))
#check_button.on_click(Check)

cluster_info_module_individual_check = VBox([cluster_info_module.children[0],
                                             cluster_info_module.children[2],
                                             cluster_info_module.children[3],
                                             cluster_info_module.children[4]])

individual_check_panel = HBox([cluster_info_module_individual_check, VBox([ID_BOX, date_select_box, check_button])])
individual_check_panel.children[1].children[2].on_click(Check)

In [ ]:
def update_viz_panel():
    global viz_panel, cluster_info_module
    if analysis_need_update['2D Visualization'] == True:
        if analysis_need_update['cluster_info_module'] == True:
            cluster_info_module = update_cluster_info_module(cluster_info_module)
            update_child(viz_panel, [0, 0], cluster_info_module.children[0])
        
        viz_panel.children[1].children[0].children[1].value = 'PCA'
        viz_panel.children[1].children[1].children[1].value = False
        
        analysis_need_update['2D Visualization'] = False
    
    if analysis_old_display['2D Visualization'] is not None:
        iplot(analysis_old_display['2D Visualization'])
        
    return viz_panel

In [ ]:
def tsne_plot(data_time, cluster_label, mark_centrid):
    name_list = data_time.index.get_level_values(0) 
    time_part = data_time.loc[:, ~data_time.columns.isin(['robust_std', 'std_group', 'cluster_label'])].copy(deep=True)
    
    try:
        data_right = pd.DataFrame(cluster_label)
        data_right = data_right[~data_right.index.duplicated('first')]
        data_right.index = data_right.index.astype('object')
        data_time.index = data_time.index.astype('object')
    except TypeError:
        print('Can\'t change datatype to object.')
    data_merge = pd.merge(data_time.loc[:, ~data_time.columns.isin(['robust_std', 'std_group', 'cluster_label'])],\
                          data_right, how='left', left_index=True, right_index=True)
    cluster_label_value = data_merge[cluster_label.name].values
    
    tsne_model = TSNE(n_components=2, n_iter_without_progress=100, random_state=0, verbose=3)
    tsne_project = tsne_model.fit_transform(time_part)
    color_list = color_assign(data_merge, cluster_label.name)
    
    data = []
    center_list = []
    center_name_list = []
    
    cluster_id_list = data_merge[cluster_label.name].unique()
    
    for i in cluster_id_list:
        index = np.where(cluster_label_value == i)
        data_temp = tsne_project[index]
        center_temp = data_temp.mean(axis=0)
        center_list.append(center_temp)
        center_name_list.append('centroid_{}'.format(i))
    
    center_list = np.array(center_list)

    # Create a trace
    trace = go.Scattergl(
        x = tsne_project[:, 0],
        y = tsne_project[:, 1],
        mode = 'markers',
        marker= dict(color=color_list),
        text = name_list,
        name = ''
    )

    data.append(trace)
        
    if mark_centrid:
        trace_center = go.Scattergl(
            x = center_list[:, 0],
            y = center_list[:, 1],
            mode = 'markers',
            marker= dict(
                        symbol = 'cross',
                        color = 'red'),
            text = center_name_list,
            name = ''
        )
    
        data.append(trace_center)
        
    
    layout = go.Layout(
        showlegend=False,
        title='Cluster Visualization - t-SNE',
        xaxis=dict(
        showgrid=False,
        zeroline=False
        ),
        
        yaxis=dict(
        showgrid=False,
        zeroline=False
        )
    )
    fig = go.Figure(data=data, layout=layout)
    return fig

In [ ]:
### PCA plot
def pca_plot(data_time, cluster_label, mark_centrid):
    name_list = data_time.index.get_level_values(0)
    time_part = data_time.loc[:, ~data_time.columns.isin(['robust_std', 'std_group', 'cluster_label'])].copy(deep=True)
    
    try:
        data_right = pd.DataFrame(cluster_label)
        data_right = data_right[~data_right.index.duplicated('first')]
        data_right.index = data_right.index.astype('object')
        data_time.index = data_time.index.astype('object')
    except TypeError:
        print('Can\'t change datatype to object.')
    
    data_merge = pd.merge(data_time.loc[:, ~data_time.columns.isin(['robust_std', 'std_group', 'cluster_label'])],\
                          data_right, how='left', left_index=True, right_index=True)
    cluster_label_value = data_merge[cluster_label.name].values
    
    PCA_model = PCA(n_components=2)
    PCA_project = PCA_model.fit_transform(time_part)
    color_list = color_assign(data_merge, cluster_label.name)
    
    data = []
    center_list = []
    center_name_list = []
    
    cluster_id_list = data_merge[cluster_label.name].unique()
    
    for i in cluster_id_list:
        index = np.where(cluster_label_value == i)
        data_temp = PCA_project[index]
        center_temp = data_temp.mean(axis=0)
        center_list.append(center_temp)
        center_name_list.append('centroid_{}'.format(i))
    
    center_list = np.array(center_list)

    # Create a trace
    trace = go.Scattergl(
        x = PCA_project[:, 0],
        y = PCA_project[:, 1],
        mode = 'markers',
        marker= dict(color=color_list),
        text = name_list,
        name = ''
    )

    data.append(trace)
        
    if mark_centrid:
        trace_center = go.Scattergl(
            x = center_list[:, 0],
            y = center_list[:, 1],
            mode = 'markers',
            marker= dict(
                        symbol = 'cross',
                        color = 'red'),
            text = center_name_list,
            name = ''
        )
    
        data.append(trace_center)
        
    
    layout = go.Layout(
        showlegend=False,
        title='Cluster Visualization - PCA',
        xaxis=dict(
        showgrid=False,
        zeroline=False
        ),
        
        yaxis=dict(
        showgrid=False,
        zeroline=False
        )
    )
    fig = go.Figure(data=data, layout=layout)
    return fig

In [ ]:
def visualize(b):
    global analysis_old_display, viz_panel
    clear_output()        
    if viz_panel.children[1].children[0].children[1].value == 'PCA':
        try:
            fig = pca_plot(data_time=mld_time.data, cluster_label=data_dim_slice[viz_panel.children[0].children[0].children[1].value],\
                           mark_centrid=viz_panel.children[1].children[1].children[1].value)
            analysis_old_display['2D Visualization'] = fig
            iplot(fig)
        except Exception as ex:
            print('Can\'t visualize the data by PCA.')
            print('Error message: {}'.format(ex))
    elif viz_panel.children[1].children[0].children[1].value == 't-SNE':
        if mld_time.data.shape[0] * mld_time.data.shape[1] > 30000:
            print('We closed t-SNE plot because this data is too big.')
        else:
            try:
                fig = tsne_plot(data_time=mld_time.data, cluster_label=data_dim_slice[viz_panel.children[0].children[0].children[1].value],\
                                mark_centrid=viz_panel.children[1].children[1].children[1].value)
                analysis_old_display['2D Visualization'] = fig
                iplot(fig)
            except Exception as ex:
                print('Can\'t visualize the data by t-SNE.')
                print('Error message: {}'.format(ex))

In [ ]:
method_label = Label('Visualization Method', layout=Layout(width='150px', margin="20px 0px 0px 200px"))
method_choice = RadioButtons(options=['PCA', 't-SNE'], value='PCA', layout=Layout(margin="20px 0px 0px 12px"))
method_box=HBox([method_label, method_choice])

center_label = Label('Mark Centroid', layout=Layout(width='100px', margin="20px 0px 0px 200px"))
center_check = Checkbox(value=False, layout=Layout(margin="20px 0px 0px 0px"))
center_box = HBox([center_label, center_check])

viz_button = Button(description='Visualize Cluster', button_style='success', layout=Layout(margin="20px 0px 0px 200px"))

cluster_info_module_viz = VBox([cluster_info_module.children[0]])

viz_panel = HBox([cluster_info_module_viz, VBox([method_box, center_box, viz_button])])
viz_panel.children[1].children[2].on_click(visualize)

# 5. Cluster INFO

In [ ]:
analysis_mapping = [update_plot_ts_panel, update_plot_dim_panel, update_cluster_info_panel, update_exception_panel,\
                    update_individual_check_panel, update_viz_panel]
analysis_tab = Tab(children=[plot_ts_panel, plot_dim_panel, cluster_info_panel, exception_panel, individual_check_panel, viz_panel],
                   _titles={0: 'Plot TS', 1: 'Plot Dim Info', 2: 'Cluster Info', 3: 'Detect Exception', 4: 'Individual Check',\
                           5: '2D Visualization'}, 
                   layout=Layout(margin='20px 0px 0px 0px'))

display(analysis_tab)

def switch_analysis(chg):
    '''Switch between processing functions.'''
    clear_output()
    children = list(analysis_tab.children)
    i = chg['new']
    children[i] = analysis_mapping[i]()
    analysis_tab.children = children

analysis_tab.observe(switch_analysis, names='selected_index')

In [ ]:
def color_assign(data_merge, label_feature):
    cluster_id_list = data_merge[label_feature].unique()
    bupu = cl.scales['9']['seq']['BuPu']
    n_color = int(math.ceil(float(len(cluster_id_list))/10)*10)
    bupuN = cl.interp(bupu, n_color)
    color_list = []
    for index_temp in data_merge.index.tolist():
        cluster_id = data_merge.loc[index_temp, label_feature]
        try:
            cluster_id = cluster_id.unique()[0]
        except AttributeError:
            pass
        color_id = np.where(np.array(cluster_id_list) == cluster_id)[0]
        if len(color_id)<1:
            color_temp = bupuN[-1]
            color_list.append(color_temp)
        else:
            color_temp = bupuN[color_id[0]]
            color_list.append(color_temp)
    return color_list

In [ ]:
def robust_mean_std(x):
    """calculate the robust mean, variance and std for a set of data
    :param x: input array provided by user
    :return location_robust: robust mean
    :return var_robust: robust variance
    :return std_robust: robust standard deviation
    """
    mcd = MinCovDet()
    try:
        mcd.fit(x)
    except ValueError:
        return x.mean(), x.var(), x.std()
    else:
        mcd.fit(x)
        location_robust = mcd.location_[0]
        var_robust = mcd.covariance_[0][0]
        std_robust = np.sqrt(var_robust)
    return location_robust, var_robust, std_robust

def cluster_robust_mean_std(data_time):
    """calculate the movement of cluster centroid and its corresponding std band
    :param data_time: return TS table for a cluster
    :return location_list: cluster centroid movement
    :return square_error_list: cluster variance band
    :return std_list: cluster standard deviation band
    """
    location_list = []
    square_error_list = []
    std_list = []
    for column in data_time.columns:
        temp = data_time.loc[:, column]
        temp = temp.reshape(-1, 1)
        location, square_error, std = robust_mean_std(temp)
        location_list.append(location)
        square_error_list.append(square_error)
        std_list.append(std)
    location_list = np.array(location_list).reshape(-1, 1)
    square_error_list = np.array(square_error_list).reshape(-1, 1)
    std_list = np.array(std_list).reshape(-1, 1)
    return location_list, square_error_list, std_list

In [ ]:
def change_into_datetime_object(mld_time):
    data_time_column = mld_time.data.columns
    first_row_name = mld_time.data.columns.name
    data_time_column_new = []
    for column_name in data_time_column:
        if not column_name in ['robust_std', 'std_group', 'cluster_label']:
            data_time_column_new.append(pd.to_datetime(column_name))
        else:
            data_time_column_new.append(column_name)
    mld_time.data.columns = data_time_column_new
    mld_time.data.columns.name = first_row_name
    
    data_original_column = mld_time.data_snapshot.columns
    first_row_name = mld_time.data_snapshot.columns.name
    data_original_column_new = []
    for column_name in data_original_column:
        if not column_name in data_dim_slice.columns.tolist()+['cluster_label']:
            data_original_column_new.append(pd.to_datetime(column_name))
        else:
            data_original_column_new.append(column_name)
    mld_time.data_snapshot.columns = data_original_column_new
    mld_time.data_snapshot.columns.name = first_row_name